In [32]:
import os, re
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import treebank
from nltk.corpus import dependency_treebank
from nltk.grammar import DependencyGrammar
from nltk.parse import DependencyGraph
import pprint
from nltk.parse.corenlp import CoreNLPDependencyParser
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
import numpy as np
import re
from numpy.linalg import norm

In [2]:
ps = PorterStemmer()
wnl = WordNetLemmatizer()
dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')

In [3]:
class Habitat:
    oid = -1
    name = ""

In [4]:
train_path = "./BioNLP-OST-2019_BB-norm_train/"
test_path = "./BioNLP-OST-2019_BB-norm_test/"
dev_path = "./BioNLP-OST-2019_BB-norm_dev/"
BB_train = os.listdir(train_path)
BB_test = os.listdir(test_path)
BB_dev = os.listdir(dev_path)
obp_file = "OntoBiotope_BioNLP-OST-2019.obo"

In [5]:
ontology = open(obp_file,encoding="utf8").read()

In [6]:
BB_train_a1 = sorted([name for name in BB_train if name.endswith("a1")])
BB_train_a2 = sorted([name for name in BB_train if name.endswith("a2")])
BB_train_txt = sorted([name for name in BB_train if name.endswith("txt")])

BB_dev_a1 = sorted([name for name in BB_dev if name.endswith("a1")])
BB_dev_a2 = sorted([name for name in BB_dev if name.endswith("a2")])
BB_dev_txt = sorted([name for name in BB_dev if name.endswith("txt")])

BB_test_a1 = sorted([name for name in BB_test if name.endswith("a1")])
BB_test_txt = sorted([name for name in BB_test if name.endswith("txt")])

In [7]:
def original_phrase(phrase):
    return phrase

In [8]:
def stem_phrase(phrase):
    #print(type(phrase))
    #print(phrase)
    
    phrase = phrase.split(" ")
    for i in range(len(phrase)):
        phrase[i] = ps.stem(phrase[i])
    phrase = " ".join(phrase)
    return phrase

In [9]:
def lemmatize_phrase(phrase):
    #print(type(phrase))
    #print(phrase)
    
    phrase = phrase.split(" ")
    for i in range(len(phrase)):
        phrase[i] = wnl.lemmatize(phrase[i])
    phrase = " ".join(phrase)
    return phrase

In [10]:
phrase_reducer = lemmatize_phrase

In [11]:
def get_headword(phrase):
    #print(phrase)
    if phrase == phrase.upper():
        return ""
    
    parses = dep_parser.parse(phrase.split())
    triples = [[(governor, dep, dependent) for governor, dep, dependent in parse.triples()] for parse in parses]
    
    try:
        if(len(triples) > 0 and len(triples[0]) > 0 and len(triples[0][0]) > 0 and len(triples[0][0][0]) > 0 and triples[0][0][0][0] is not None):
            return triples[0][0][0][0]
        else:
            return phrase
    except:
        return phrase
    
    

In [12]:
'asdsa-asdas asda-asda'.replace('-', ' ')

'asdsa asdas asda asda'

In [13]:
def add_documents(txt, processor):
    document = []
    txt = processor(txt)
    sentence_list = sent_tokenize(txt)
    
    for sentence in sentence_list:
        sentence = sentence.replace('-', ' ')
        word_list = word_tokenize(sentence)
        word_list = [word for word in word_list if word not in stopwords.words("english") and word.isalnum()]
        for i in range(len(word_list)):
            if word_list[i].isnumeric():
                word_list[i] = "##number##"
            if word_list[i] != word_list[i].upper():
                word_list[i] = processor(word_list[i].lower())
             
        document.append(word_list)
    #print(document)
    return document

In [14]:
documents = []
for i in range(len(BB_train_txt)):
    txt = open(train_path + BB_train_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)
for i in range(len(BB_dev_txt)):
    txt = open(dev_path + BB_dev_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)
for i in range(len(BB_test_txt)):
    txt = open(test_path + BB_test_txt[i],encoding="utf8").read()
    documents += add_documents(txt, phrase_reducer)

In [15]:
model = Word2Vec(documents, min_count=1)

In [16]:
phrase_reducer('Nares')

'Nares'

In [17]:
model['naris']

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 3.1863973e-03, -8.6706085e-04, -8.2898606e-04, -1.7189150e-03,
        4.3258346e-03, -3.6904532e-03,  3.6472126e-03, -3.2985455e-03,
       -6.5725960e-04, -1.2635067e-03, -1.3165143e-03, -2.2431721e-03,
        3.2759970e-03, -1.9173056e-03, -8.9343230e-04,  4.4838646e-03,
        5.1806481e-03, -3.8529499e-03,  7.7804533e-04,  3.2806129e-05,
        4.8146053e-04,  3.9780750e-03,  7.7236327e-04, -1.5536738e-05,
        2.9028915e-03, -4.4611837e-03,  2.8537540e-03,  3.0164961e-03,
       -4.7025708e-03,  5.1002013e-03,  2.2748029e-03,  3.5821716e-03,
       -1.8947700e-03,  5.9327050e-03, -5.4882537e-03, -1.2934937e-03,
        5.2857855e-03,  4.8262714e-03,  4.9997787e-03, -4.1296934e-03,
       -2.4528129e-03, -5.8493433e-03, -3.8581868e-03, -2.4266313e-03,
       -9.4540988e-04, -1.3471590e-03, -1.5502382e-03, -2.9151235e-03,
       -9.6646883e-04, -1.4263766e-03,  2.2284840e-03, -3.3520774e-03,
       -1.7008012e-03,  1.6830319e-03,  2.0365522e-03, -3.4813050e-03,
      

In [18]:
len(model.wv.vocab)

5537

In [19]:
type(list(model.wv.vocab))

list

In [20]:
len(model[list(model.wv.vocab)[0]])

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


100

In [21]:
def get_phrase_embedding(phrase, model, processor):
    original_phrase = phrase
    phrase = phrase.replace('-', ' ')
    phrase = nltk.word_tokenize(phrase)
    
    phrase = [processor(word) for word in phrase if word.lower() not in stopwords.words("english") and word.isalnum()]
    for i in range(len(phrase)):
        if phrase[i].isnumeric():
            phrase[i] = "##number##"
    
    phrase_size = len(phrase)
    vec_size = len(model[list(model.wv.vocab)[0]])
    embedded_sum = np.zeros(vec_size)
    if(phrase_size == 0):
        return -1
    for word in phrase:
        if word not in list(model.wv.vocab):
            return -1
        embedded_sum = embedded_sum + np.array(model[word])
    embedded_sum = embedded_sum / phrase_size
    
    return embedded_sum
    

In [22]:
def generate_habitat_map(processor, model):
    habitat_list = ontology.split("\n\n[Term]\n")[1:]
    habitat_map = {}
    habitat_map_originals = {}
    habitat_embeddings = {}
    for h in habitat_list:
        #new_habitat = Habitat()
        h = h.split("\n")
        oid = re.findall(r"(?<=id: OBT:)[0-9]+", h[0])[0]
        name = (re.findall(r"(?<=name: ).+", h[1])[0])
        original_name = name
        if name != name.upper():
            name = name.lower()
            name = processor(name)
    
        habitat_map[name] = oid
        habitat_map_originals[oid] = name
        embedding = get_phrase_embedding(name, model, processor)
        if type(embedding) == type(np.array([1])):
            habitat_embeddings[name] = embedding
    
        # word2vec buraya eklenmeli 
        for h_line in h:
            if "synonym" in h_line:
                synonym_name = (re.findall(r"(?<=synonym: \").+(?=\")", h_line)[0])
                if synonym_name != synonym_name.upper():
                    synonym_name = synonym_name.lower()
                    synonym_name = processor(synonym_name)
                if "EXACT" in h_line and synonym_name not in habitat_map:
                    habitat_map[synonym_name] = oid
                    if type(embedding) == type(np.array([1])):
                        habitat_embeddings[synonym_name] = embedding
                    
    return habitat_map, habitat_map_originals, habitat_embeddings

In [23]:
habitat_map, habitat_map_originals, habitat_embeddings = generate_habitat_map(phrase_reducer, model)

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [24]:
len(habitat_map)

4000

In [25]:
len(habitat_embeddings)

938

In [26]:
def add_habitats(a1, a2, processor, model):
    a2 = a2.split("\n")
    a1 = a1.split("\n")
    
    
    for annotation in a2:
        a1_line = 0
        t = ""
        oid = ""
        #print(annotation)
        if "OntoBiotope" in annotation:
            t = re.findall(r"(?<=Annotation:)T[0-9]+", annotation)[0]
            oid = re.findall(r"(?<=Referent:OBT:)[0-9]+", annotation)[0]
            named_entity = ""
            
            while t not in a1[a1_line] and a1_line < len(a1)-1:
                a1_line += 1
            if "Habitat" in a1[a1_line]: 
                named_entity = (a1[a1_line].split("\t")[2])
                named_entity = named_entity.replace('-', ' ')
                if named_entity != named_entity.upper():
                    named_entity = named_entity.lower()
                    named_entity = processor(named_entity)
                if named_entity not in habitat_map:
                    habitat_map[named_entity] = oid
                
                if habitat_map_originals[oid] not in habitat_embeddings:
                    habitat_embeddings[habitat_map_originals[oid]] = []
                
                if type(habitat_embeddings[habitat_map_originals[oid]]) == list:
                    emb = get_phrase_embedding(named_entity, model, processor)

                    habitat_embeddings[habitat_map_originals[oid]].append(emb)

In [27]:
for i in range(len(BB_train_txt)):    
    a1 = open(train_path + BB_train_a1[i],encoding="utf8").read()
    a2 = open(train_path + BB_train_a2[i],encoding="utf8").read()
    add_habitats(a1, a2, phrase_reducer, model)

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [28]:
for key in habitat_embeddings:
    if type(habitat_embeddings[key]) == list:
        vec_size = len(habitat_embeddings[key][0])
        temp_vec = np.zeros(vec_size)
        for vec in habitat_embeddings[key]:
            temp_vec += vec
        temp_vec /= len(habitat_embeddings[key])
        habitat_embeddings[key] = temp_vec

In [29]:
len(habitat_map_originals)

3602

In [30]:
habitat_embeddings

{'fermented cheese': array([ 9.39517263e-02,  4.03038976e-02,  6.04095901e-02,  6.56619514e-02,
         4.92899844e-02, -8.10975456e-02,  9.55706574e-02, -6.67793546e-02,
         5.93069787e-02, -5.42101327e-02, -1.68208853e-01,  8.22127995e-03,
        -7.31625785e-02,  3.71060844e-02,  5.19324504e-03,  2.06778776e-02,
         8.49254061e-02,  9.47670694e-02,  3.18022158e-02,  4.87449928e-03,
        -9.77489934e-03,  1.04312946e-01, -1.50417028e-01,  4.42908541e-02,
        -9.85023100e-04, -7.88722159e-02, -5.06456872e-02, -2.95744585e-02,
         2.51055472e-02,  5.28272176e-02,  4.35058931e-02, -1.22020165e-02,
         1.00308486e-01,  1.22381187e-01, -9.89879756e-02, -1.21695082e-01,
         7.15213581e-02,  4.88819424e-02,  1.15272488e-01, -1.54371699e-02,
        -4.36680197e-02, -1.17694066e-01, -5.10223056e-02,  1.43554460e-02,
         5.96097170e-02, -6.04741853e-02,  4.65182832e-02,  1.20982843e-01,
         3.37802118e-02, -3.92920217e-02, -8.76462897e-02, -2.089649

In [31]:
def predict_habitats_exact_matching(a1, txt, a1_name, processor, pred_file_name):
    pred_file = open(pred_file_name, "w")
    cands = 0
    matches = 0
    match_list = {}
    a1 = a1.split("\n")
    count = 1
    for line in a1:
        found = False
        if "Habitat" in line:
            cands += 1
            name = (line.split("\t")[2])
            named_entity = name
            if name != name.upper():
                name = name.lower()
                named_entity = processor(name)
            if named_entity in habitat_map:
                matches += 1
                match = name + " - " + habitat_map[named_entity] + "---exact"
                match_list[line.split("\t")[0]] = [habitat_map[named_entity], named_entity, "exact"]
                found = True
            else:
                #print(name)
                pre_named_entity = get_headword(name)
                #print(pre_named_entity)
                named_entity = processor(pre_named_entity)
                if named_entity in habitat_map:
                    matches += 1
                    match = name + " - " + habitat_map[named_entity] + "---headwordexact"
                    print(match)
                    match_list[line.split("\t")[0]] = [habitat_map[named_entity], name, "headwordexact", named_entity]
                    found = True
        if found:
            pred_file.write("N" + str(count) +"\tOntoBiotope Annotation:" + line.split("\t")[0] + " Referent:OBT:" + match_list[line.split("\t")[0]][0] + "\n")
            count += 1
    #print(a1_name)
    #print("Out of", cands, "candidates,", matches, "matches found")
    #print()
    pred_file.close()
    return match_list

In [37]:
def predict_habitats(a1, txt, a1_name, processor, pred_file_name, model):
    pred_file = open(pred_file_name, "w")
    cands = 0
    matches = 0
    match_list = {}
    a1 = a1.split("\n")
    count = 1
    for line in a1:
        found = False
        if "Habitat" in line:
            cands += 1
            name = (line.split("\t")[2])
            name = name.replace("-", " ")
            named_entity = name
            entity_vec = get_phrase_embedding(named_entity, model, processor)
            if type(entity_vec) != int:
                largest_prod = np.dot(entity_vec, habitat_embeddings[list(habitat_embeddings)[0]])/(norm(entity_vec)*norm(habitat_embeddings[list(habitat_embeddings)[0]]))
                match_id = habitat_map[list(habitat_embeddings)[0]]
                for habitat in habitat_embeddings:
                    prod = np.dot(entity_vec,habitat_embeddings[habitat])/(norm(entity_vec)*norm(habitat_embeddings[habitat]))
                    if prod > largest_prod:
                        largest_prod = prod
                        match_id = habitat_map[habitat]
                
                match_list[line.split("\t")[0]] = [match_id, named_entity, "w2v"]
                    
        #if found:
            #pred_file.write("N" + str(count) +"\tOntoBiotope Annotation:" + line.split("\t")[0] + " Referent:OBT:" + match_list[line.split("\t")[0]][0] + "\n")
            #count += 1
    #print(a1_name)
    #print("Out of", cands, "candidates,", matches, "matches found")
    #print()
    pred_file.close()
    return match_list

In [38]:
annotations_test = {}
for i in range(len(BB_dev_txt)):
    a1 = open(dev_path + BB_dev_a1[i],encoding="utf8").read()
    txt = open(dev_path + BB_dev_txt[i],encoding="utf8").read()
    pred_file_name = "./dev_preds/" + BB_dev_a2[i]
    annotations_test[BB_dev_a1[i]] = predict_habitats(a1, txt, BB_dev_a1[i], phrase_reducer, pred_file_name, model)

C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
C:\Users\eges9\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [40]:
annotations_test

{'BB-norm-10496597.a1': {'T3': ['000640', 'gastric', 'w2v'],
  'T4': ['001577', 'gastric mucosa', 'w2v'],
  'T5': ['000640', 'gastric mucosal associated lymphoid tissue', 'w2v'],
  'T6': ['001654', 'gastric mucosal', 'w2v'],
  'T7': ['000640', 'gastric', 'w2v'],
  'T8': ['000424', 'MALT', 'w2v'],
  'T9': ['000061', 'monoclonal B cells', 'w2v'],
  'T11': ['000640', 'gastric', 'w2v'],
  'T12': ['001129', 'gastric MALT', 'w2v'],
  'T13': ['003220', 'patients with atypical lymphoid infiltrates', 'w2v'],
  'T14': ['003220', 'patients with low grade MALT lymphoma', 'w2v'],
  'T16': ['000268',
   'patients with high grade primary gastric lymphoma',
   'w2v'],
  'T17': ['000640', 'gastric', 'w2v'],
  'T18': ['000424', 'MALT', 'w2v'],
  'T20': ['001182', 'CD20', 'w2v'],
  'T21': ['000100', 'CD3', 'w2v'],
  'T22': ['000424', 'MALT', 'w2v'],
  'T23': ['003220', 'patients with chronic active gastritis', 'w2v'],
  'T24': ['000061', 'CD20 positive cells', 'w2v'],
  'T25': ['001335', 'lymphocytic', '

In [41]:
def test_habitats(a1, a2, processor):
    
    a2 = a2.split("\n")
    a1 = a1.split("\n")
    match_list = {}
    for annotation in a2:
        a1_line = 0
        t = ""
        oid = ""
        #print(annotation)
        if "OntoBiotope" in annotation:
            t = re.findall(r"(?<=Annotation:)T[0-9]+", annotation)[0]
            oid = re.findall(r"(?<=Referent:OBT:)[0-9]+", annotation)[0]
            named_entity = ""
            while t not in a1[a1_line] and a1_line < len(a1)-1:
                a1_line += 1
            if "Habitat" in a1[a1_line]:
                named_entity = (a1[a1_line].split("\t")[2])
                if named_entity != named_entity.upper():
                    named_entity = named_entity.lower()
                    named_entity = processor(named_entity)
                match_list[t] = [oid, named_entity]
    return match_list

In [42]:
dev_a2 = {}
for i in range(len(BB_dev_txt)):
    a1 = open(dev_path + BB_dev_a1[i],encoding="utf8").read()
    a2 = open(dev_path + BB_dev_a2[i],encoding="utf8").read()
    dev_a2[BB_dev_a1[i]] = test_habitats(a1,a2, phrase_reducer)

In [43]:
true_pred_count = 0
false_pred_count = 0
total_count = 0
for key in dev_a2:
    """print()
    print(key)
    print(dev_a2[key])
    print("**************")"""
    total_count += len(dev_a2[key])
    if key in annotations_test:
        for pred in annotations_test[key]:            
            if annotations_test[key][pred][0] == dev_a2[key][pred][0]:
                true_pred_count += 1
            else:
                print(key)
                print("Prediction:", annotations_test[key][pred]," Real:", dev_a2[key][pred])
                print("Prediction:", habitat_map_originals[annotations_test[key][pred][0]], " Real:", habitat_map_originals[dev_a2[key][pred][0]])
                false_pred_count += 1

BB-norm-10496597.a1
Prediction: ['000640', 'gastric', 'w2v']  Real: ['001792', 'gastric']
Prediction: gastric acid  Real: stomach
BB-norm-10496597.a1
Prediction: ['000640', 'gastric mucosal associated lymphoid tissue', 'w2v']  Real: ['000334', 'gastric mucosal-associated lymphoid tissue']
Prediction: gastric acid  Real: lymphatic system part
BB-norm-10496597.a1
Prediction: ['001654', 'gastric mucosal', 'w2v']  Real: ['001577', 'gastric mucosal']
Prediction: mucosal surface  Real: gastric mucosa
BB-norm-10496597.a1
Prediction: ['000640', 'gastric', 'w2v']  Real: ['001792', 'gastric']
Prediction: gastric acid  Real: stomach
BB-norm-10496597.a1
Prediction: ['000424', 'MALT', 'w2v']  Real: ['000334', 'MALT']
Prediction: skin part  Real: lymphatic system part
BB-norm-10496597.a1
Prediction: ['000061', 'monoclonal B cells', 'w2v']  Real: ['001623', 'monoclonal b cell']
Prediction: cell  Real: lymphocyte
BB-norm-10496597.a1
Prediction: ['000640', 'gastric', 'w2v']  Real: ['001792', 'gastric']

Prediction: non-immune serum  Real: blood serum
BB-norm-9521147.a1
Prediction: ['000323', 'microtiter plates', 'w2v']  Real: ['000103', 'microtiter plate']
Prediction: industrial product  Real: laboratory equipment
BB-norm-9553794.a1
Prediction: ['000268', 'vacuum  packed cold smoked salmon stored at 5 degrees C', 'w2v']  Real: ['002593', 'vacuum- packed cold-smoked salmon stored at 5 degree c']
Prediction: enterocyte  Real: salmon meat
BB-norm-9553794.a1
Prediction: ['000268', 'modified atmosphere packed cold smoked salmon stored at 5 degrees C', 'w2v']  Real: ['001774', 'modified-atmosphere-packed cold-smoked salmon stored at 5 degree c']
Prediction: enterocyte  Real: smoked food
BB-norm-9553794.a1
Prediction: ['001774', 'spoiled cold smoked salmon', 'w2v']  Real: ['002593', 'spoiled cold-smoked salmon']
Prediction: smoked food  Real: salmon meat
BB-norm-9553794.a1
Prediction: ['002300', 'vacuum packed cold smoked salmon', 'w2v']  Real: ['001774', 'vacuum-packed cold-smoked salmon']


In [44]:
recall = true_pred_count/total_count
recall

0.3081967213114754

In [45]:
precision = true_pred_count/(true_pred_count + false_pred_count)
precision

0.4060475161987041

## Denemeler